## About this notebook:

This notebook shows you how to use Napari to create a mask to exclude big areas of an image from analysis. Excluded regions can be marked using shapes layer available in napari (https://napari.org/howtos/layers/shapes.html).

Input:
- aligned tiffs from selected well
- dat frame

Output:
- mask for a given well (all channels together)

Remember to save your mask before closing the napari window!!!

## Fill in info about the experiment to process

In [14]:
# pathway to a directory with data frames (ex. df)
path_df = r''

# pathway to aligned tiffs (ex. aligned_tiffs)
path_tiffs = r''

# pathway to save punch masks (ex. masks)
path_save = r''

# choose the well for processing (single well)
selected_well = 'A4'

# set to True if you want to add regions to an existing mask
use_existing = True

## Prepare for processing

In [15]:
import os
import pickle5 as pickle
import pandas as pd
from tifffile import imread
from skimage.io import imsave
import napari
from magicgui.widgets import Container,Label,Dropdown
from magicgui import magicgui

In [ ]:
# create directory for saving data frames if needed
try:
    os.mkdir(path_save)
    print('Directory for saving aligned tiffs created.')
except:
    print('Directory not created.')

In [16]:
# open data frame

df_name = f'df_{selected_well}.pkl'

with open(os.path.join(path_df,df_name), "rb") as fh:
  df = pickle.load(fh)

# open starting image
im_path = os.path.join(path_tiffs,selected_well,df.aligned_file_name[0])
im = imread(im_path)

In [17]:
# try reading the mask
if use_existing:
    
    try:
        mask_path = os.path.join(path_save,f'mask_{selected_well}.pkl')
        mask = pickle.load( open( mask_path, "rb" ) )
        print('Existing mask found.')
    except:
        print('Mask not found')

In [18]:
# add saving button
def save_mask(viewer: napari.Viewer):
    
    mask = viewer.layers['mask'].data
    mask_path = os.path.join(path_save,f'mask_{selected_well}.pkl')
    
    pickle.dump(mask, open( mask_path, "wb" ) )
    viewer.status = 'Data has been saved.'

save_data = magicgui(save_mask, call_button='Save Mask')

In [19]:
# add selection of images
channel_list = ['_'.join([str(int(x)).zfill(2),y]) for x,y in zip(df.nameRound,df.signal.to_list())]
df['round_channel'] = channel_list

def channel_change(val_coming: str):
    
    global viewer
    
    alig_file = df.loc[df.round_channel == val_coming,'aligned_file_name'].to_list()[0]

    im_path = os.path.join(path_tiffs,selected_well,alig_file)
    im = imread(im_path)

    viewer.layers[1].data = im
    viewer.layers[1].name = df.loc[df.round_channel == val_coming,'signal'].to_list()[0]
    
    viewer.status = 'New channel opened.'


widget_channel = Dropdown(choices=channel_list)
widget_channel.changed.connect(channel_change)

<function __main__.channel_change(val_coming: str)>

## Open napari to draw masks

In [20]:
viewer = napari.Viewer()

viewer.add_shapes(name = 'mask',face_color='red')

if use_existing:
    
    viewer.layers['mask'].data = mask
    
viewer.add_image(im,blending = 'additive',contrast_limits=[0,4095],name=df.signal[0])

viewer.window.add_dock_widget(widget_channel,area='left',name='Choose channel')
viewer.window.add_dock_widget(save_data,area='left',name='Save Mask')